In [51]:
import matplotlib.pyplot as plt
import numpy
import pandas
import plotly
import langdetect 


In [52]:
DATA_URL = '/metadata/metadata.csv'


In [53]:
datadf = pandas.read_csv(DATA_URL)


In [4]:
print(datadf.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51078 entries, 0 to 51077
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   cord_uid                     51078 non-null  object 
 1   sha                          38022 non-null  object 
 2   source_x                     51078 non-null  object 
 3   title                        50920 non-null  object 
 4   doi                          47741 non-null  object 
 5   pmcid                        41082 non-null  object 
 6   pubmed_id                    37861 non-null  float64
 7   license                      51078 non-null  object 
 8   abstract                     42352 non-null  object 
 9   publish_time                 51070 non-null  object 
 10  authors                      48891 non-null  object 
 11  journal                      46368 non-null  object 
 12  Microsoft Academic Paper ID  964 non-null    float64
 13  WHO #Covidence  

In [5]:
datadf.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,PMC,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,PMC140314,12525263.0,no-cc,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",BMC Public Health,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
1,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,PMC,Discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,PMC156578,12734001.0,no-cc,Recent analyses of human pathogens have reveal...,2003-04-28,"Disotell, Todd R",Genome Biol,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
2,le0ogx1s,NaN,PMC,A new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,PMC193621,12844350.0,no-cc,"The army of the men of death, in John Bunyan's...",2003-06-27,"Petsko, Gregory A",Genome Biol,NaN,NaN,False,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,PMC,Association of HLA class I with severe acute r...,10.1186/1471-2350-4-9,PMC212558,12969506.0,no-cc,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,"Lin, Marie; Tseng, Hsiang-Kuang; Trejaut, Jean...",BMC Med Genet,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
4,0qaoam29,5b68a553a7cbbea13472721cd1ad617d42b40c26,PMC,A double epidemic model for the SARS propagation,10.1186/1471-2334-3-19,PMC222908,12964944.0,no-cc,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,"Ng, Tuen Wai; Turinici, Gabriel; Danchin, Antoine",BMC Infect Dis,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...


In [54]:
%%time
def detect_language(text):
  try:
    return langdetect.detect(text)
  except:
    return None

datadf['language'] = datadf['title'].apply(detect_language)

CPU times: total: 4min 48s
Wall time: 5min 17s


In [41]:
print('Languages: %s' % datadf['language'].unique())
print(datadf[['language', 'cord_uid']].groupby('language')
                                      .count()
                                      .sort_values('cord_uid', ascending=False))
print('Licences: %s...' % datadf['license'].unique())
print(datadf[['license', 'cord_uid']].groupby('license')
                                     .count()
                                     .sort_values('cord_uid', ascending=False))

Languages: ['en' 'ro' 'lt' 'ca' 'fr' 'it' 'id' 'de' 'tl' 'nl' 'es' 'da' 'sw' 'af'
 'et' 'cy' 'pt' 'sv' 'no' 'zh-cn' 'ko' 'hu' 'fi' 'vi' 'el' 'sl' 'pl' 'sq'
 'lv' 'so' 'sk' None 'hr' 'tr']
          cord_uid
language          
en           47906
fr             773
de             426
es             401
it             335
ca             206
ro             164
nl             132
af              90
et              73
pt              58
tl              51
id              45
lt              45
no              44
da              39
vi              22
fi              20
cy              14
sl              10
so              10
sv              10
pl               8
sw               8
hu               7
ko               6
hr               5
tr               4
lv               3
zh-cn            2
sq               1
el               1
sk               1
Licences: ['no-cc' 'cc-by' 'cc0' 'unk' 'cc-by-nc' 'cc-by-nc-nd' 'cc-by-nc-sa'
 'cc-by-nd' 'els-covid' 'biorxiv' 'medrvix' 'cc-by-sa' 'pd']...
     

In [55]:
LANGUAGE = 'en'

docdf = datadf.loc[datadf['language'] == LANGUAGE, ['cord_uid', 'publish_time', 'title', 'authors', 'abstract','url']] \
              .rename(columns=dict(cord_uid='id', publish_time='date', abstract='text'))
docdf = docdf.groupby('id').first().reset_index()
docdf.sort_values('date', ascending=False, inplace=True)
docdf.dropna(inplace=True)


In [72]:
urls = datadf['url']

In [89]:
urls = list(urls)

In [92]:
with open('url.txt', 'rb') as fp:
        titles = json.load(fp)

In [8]:

print(docdf.info())
docdf.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40209 entries, 9340 to 173
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       40209 non-null  object
 1   date     40209 non-null  object
 2   title    40209 non-null  object
 3   authors  40209 non-null  object
 4   text     40209 non-null  object
dtypes: object(5)
memory usage: 1.8+ MB
None


,id,date,title,authors,text
9340,6zld1n9n,2020-12-31,Chapter 27 Emergence and Reemergence of Viral ...,"El Amri, Hamid; Boukharta, Mohamed; Zakham, Fa...",Abstract The unpredictable emergence of new zo...
36179,r5va3y1n,2020-12-31,Chapter 48 Mucosal Vaccine Development for Vet...,"Wilson, Heather L.; Gerdts, Volker; Babiuk, Lo...",Abstract Because most pathogens and food antig...
40072,u4i5tmey,2020-12-31,Chapter 40 Waterborne Transmission of Enteric ...,"Bouseettine, Rihab; Hassou, Najwa; Bessi, Hlim...",Abstract Viruses of human or animal origin can...
22001,ggac3kl0,2020-12-31,13 Nanomaterial-based sensors,"Arduini, Fabiana; Cinti, Stefano; Scognamiglio...",Abstract This chapter deals with the advanceme...
11822,8sklix0x,2020-12-31,Chapter 12 Next generation agents (synthetic a...,"Sharma, Anshula; Gupta, Gaganjot; Ahmad, Tawse...",Abstract Biological weapons are highly contagi...


In [ ]:
docdf['text']

In [29]:
import gensim
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\el
[nltk_data]     ouatiq\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [30]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to C:\Users\el
[nltk_data]     ouatiq\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [21]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer("english")
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess2(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [46]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [48]:
processed_docs2 = [preprocess2(doc) for doc in docdf['text']]

In [52]:
import json
with open("proc.txt", "w") as f:
    json.dump(processed_docs2, f)

In [53]:
with open("proc.txt") as f:
    lst = json.load(f)

In [19]:
from gensim.corpora import Dictionary

In [16]:
dictionary = Dictionary(processed_docs2)

In [17]:
# Convert the preprocessed documents into a bag-of-words representation
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs2]

In [15]:
import json 
with open("proc.txt") as f:
    processed_docs2 = json.load(f)

In [18]:
import gensim
# Create a TF-IDF model from the bag-of-words representation
tfidf = gensim.models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [37]:
# Create an LSI model from the TF-IDF representation
lsi_model2 = gensim.models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)

2023-02-20 17:55:52.444 using serial LSI version on this node
2023-02-20 17:55:52.634 updating model with new documents
2023-02-20 17:56:00.827 preparing a new chunk of documents
2023-02-20 17:56:01.447 using 100 extra samples and 2 power iterations
2023-02-20 17:56:01.451 1st phase: constructing (51575, 102) action matrix
2023-02-20 17:56:06.277 orthonormalizing (51575, 102) action matrix
2023-02-20 17:56:11.619 2nd phase: running dense svd on (102, 20000) matrix
2023-02-20 17:56:14.003 computing the final decomposition
2023-02-20 17:56:14.035 keeping 2 factors (discarding 81.872% of energy spectrum)
2023-02-20 17:56:14.225 processed documents up to #20000
2023-02-20 17:56:14.718 topic #0(22.442): 0.200*"mer" + 0.169*"patient" + 0.159*"covid" + 0.135*"case" + 0.127*"respiratori" + 0.127*"infect" + 0.127*"virus" + 0.117*"vaccin" + 0.111*"diseas" + 0.108*"sar"
2023-02-20 17:56:14.728 topic #1(13.462): -0.352*"covid" + -0.240*"patient" + -0.210*"case" + 0.185*"cell" + 0.168*"protein" + 0

In [34]:
lsi_model2.save("lsi.model")

2023-02-20 17:49:48.492 Projection lifecycle event {'fname_or_handle': 'lsi.model.projection', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-02-20T17:49:48.426740', 'gensim': '4.3.0', 'python': '3.9.0 (tags/v3.9.0:9cf6752, Oct  5 2020, 15:34:40) [MSC v.1927 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'saving'}
2023-02-20 17:49:50.137 saved lsi.model.projection
2023-02-20 17:49:50.140 LsiModel lifecycle event {'fname_or_handle': 'lsi.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': ['projection', 'dispatcher'], 'datetime': '2023-02-20T17:49:50.140501', 'gensim': '4.3.0', 'python': '3.9.0 (tags/v3.9.0:9cf6752, Oct  5 2020, 15:34:40) [MSC v.1927 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'saving'}
2023-02-20 17:49:50.337 not storing attribute projection
2023-02-20 17:49:50.338 not storing attribute dispatcher
2023-02-20 17:50:01.595 saved lsi.model


In [ ]:
from gensim.models import LsiModel
lsi_model = LsiModel.load("lsi.model")

In [37]:
# Define the query what is difference between sars  and corona virus.?
query = ""

In [38]:
# Preprocess the query
query_processed = preprocess2(query)


In [39]:
# Convert the preprocessed query into a bag-of-words representation
bow_query = dictionary.doc2bow(query_processed)


In [40]:
# Create a TF-IDF representation of the preprocessed query
tfidf_query = tfidf[bow_query]


In [42]:
# Get the LSI representation of the preprocessed query
lsi_query = lsi_model[tfidf_query]


In [33]:
from gensim import similarities

In [35]:
from gensim import corpora
corpus_load = corpora.MmCorpus('BoW_corpus.mm')

In [47]:
# calculate the cosine similarity between the query and each document in the corpus
index = similarities.MatrixSimilarity(lsi_model[corpus_load])
sims = index[lsi_query]

In [44]:

corpora.MmCorpus.serialize("BoW_corpus.mm", bow_corpus)

2023-02-20 18:08:32.921 storing corpus in Matrix Market format to BoW_corpus.mm
2023-02-20 18:08:32.924 saving sparse matrix to BoW_corpus.mm
2023-02-20 18:08:32.925 PROGRESS: saving document #0
2023-02-20 18:08:33.427 PROGRESS: saving document #1000
2023-02-20 18:08:33.792 PROGRESS: saving document #2000
2023-02-20 18:08:34.042 PROGRESS: saving document #3000
2023-02-20 18:08:34.398 PROGRESS: saving document #4000
2023-02-20 18:08:34.744 PROGRESS: saving document #5000
2023-02-20 18:08:35.023 PROGRESS: saving document #6000
2023-02-20 18:08:35.281 PROGRESS: saving document #7000
2023-02-20 18:08:35.521 PROGRESS: saving document #8000
2023-02-20 18:08:35.771 PROGRESS: saving document #9000
2023-02-20 18:08:35.991 PROGRESS: saving document #10000
2023-02-20 18:08:36.274 PROGRESS: saving document #11000
2023-02-20 18:08:36.519 PROGRESS: saving document #12000
2023-02-20 18:08:36.935 PROGRESS: saving document #13000
2023-02-20 18:08:37.169 PROGRESS: saving document #14000
2023-02-20 18:08

In [120]:
sims_sorted_with_urls = [(index, score, urls[index]) for index, score in sorted(enumerate(sims), key=lambda item: -item[1])]

KeyError: 40161

In [108]:
sims_sorted_with_urls[0]

(6075, 1.0, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4766874/')

In [137]:
sims_sorted_with_urls = sims_sorted_with_urls[0:20]

In [138]:
for index, score, url in sims_sorted_with_urls:
    print(f"{url} \n {score}")

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4766874/ 
 1.0
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5535864/ 
 1.0
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7123565/ 
 1.0
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7120100/ 
 1.0
https://doi.org/10.1016/b978-0-7020-4061-0.00004-2 
 1.0
https://doi.org/10.1016/s0091-6749(03)81027-8 
 1.0
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC112386/ 
 1.0
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2396482/ 
 0.9999999403953552
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4714422/ 
 0.9999999403953552
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4803184/ 
 0.9999999403953552
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6487183/ 
 0.9999999403953552
https://doi.org/10.1101/2020.04.02.022186 
 0.9999999403953552
https://doi.org/10.1016/j.smim.2004.02.003 
 0.9999999403953552
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6143185/ 
 0.9999998807907104
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7107947/ 
 0.9999998807907104
https://www.ncb